In [18]:
import pandas as pd

In [19]:
# 1. 데이터 불러오기
share_raw = pd.read_excel("E:/Coding/TMDL/전국오염원조사/단위유역별 점유율.xlsx")
region_info = pd.read_excel("E:/Coding/TMDL/전국오염원조사/동리별_유역현황.xlsx")

In [20]:
# 2. 동리별 유역현황에서 필요한 열만 선택
region_info = region_info[['동리코드', '단위유역', '점유율']]

region_info

,동리코드,단위유역,점유율
0,춘천시 봉의동,북한C,100.0
1,춘천시 요선동,북한C,100.0
2,춘천시 낙원동,북한C,100.0
3,춘천시 중앙로1가,북한C,100.0
4,춘천시 중앙로2가,북한C,100.0
...,...,...,...
1362,양양군 강현면 장산리,기타,100.0
1363,양양군 강현면 전진리,기타,100.0
1364,양양군 강현면 주청리,기타,100.0
1365,양양군 강현면 적은리,기타,100.0


In [21]:
# 3. share_원본의 계열 컬럼(생활계 ~ 유원지) 추출
share_cols = share_raw.loc[:, '생활계':'유원지'].columns.tolist()

In [22]:
# 4. 점유율 자료의 동리코드 수정

# '읍면동'과 '동리'가 같은 경우 '동리'의 값 삭제
share_raw.loc[share_raw['읍면동'] == share_raw['동리'], '동리'] = ''

# '동리코드' 열 삭제
share_raw.drop(columns=['동리코드'], inplace=True)

# 동리코드 수정
share_raw.insert(
    0,
    '동리코드',
    (
        share_raw['시군'].fillna('') + ' ' +
        share_raw['읍면동'].fillna('') + ' ' +
        share_raw['동리'].fillna('')
    ).str.strip()
)

share_raw

,동리코드,단위유역,소유역,시도,시군,읍면동,동리,생활계,축산계,산업계,...,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,면적(㎢),면적점유율(%)
0,평창군 대관령면 유천리,골지A,골지A01,강원도,평창군,대관령면,유천리,0.000000,0.000000,0.0,...,0.558518,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0422,0.001298
1,평창군 대관령면 병내리,골지A,골지A01,강원도,평창군,대관령면,병내리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0452,0.001184
2,평창군 대관령면 차항리,골지A,골지A01,강원도,평창군,대관령면,차항리,99.446379,99.446379,0.0,...,98.260804,100.000000,0.0,100.0,99.054376,100.0,100.000000,0.0,25.7825,0.991963
3,평창군 대관령면 횡계리,골지A,골지A01,강원도,평창군,대관령면,횡계리,100.000000,100.000000,100.0,...,100.000000,100.000000,100.0,100.0,100.000000,100.0,100.000000,100.0,61.8261,0.999173
4,평창군 대관령면 수하리,골지A,골지A01,강원도,평창군,대관령면,수하리,48.231325,48.231325,0.0,...,43.786371,33.222964,0.0,100.0,27.344476,0.0,42.631997,0.0,4.1141,0.129039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,영월군 수주면 무릉리,주천A,주천A11,강원도,영월군,수주면,무릉리,81.727353,81.727353,100.0,...,96.940144,83.169701,100.0,100.0,75.651966,0.0,100.000000,0.0,0.0000,1.000000
2409,영월군 수주면 법흥리,주천A,주천A11,강원도,영월군,수주면,법흥리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.000000
2410,영월군 수주면 도원리,주천A,주천A11,강원도,영월군,수주면,도원리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.000000
2411,영월군 수주면 무릉리,평창A,평창A09,강원도,영월군,수주면,무릉리,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0000,0.000000


In [23]:
# 5. 그룹화하여 합계 계산 및 100으로 나누어 비율로 변환
share = (
    share_raw
    .groupby(['동리코드', '단위유역', '시군'], as_index=False)[share_cols]
    .sum()
)

share[share_cols] = share[share_cols] / 100

share

,동리코드,단위유역,시군,생활계,축산계,산업계,양식계,전,답,임야,대지,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지
0,강릉시 강동면 모전리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,강릉시 강동면 산성우리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,강릉시 강동면 상시동리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,강릉시 강동면 심곡리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,강릉시 강동면 안인리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,횡성군 횡성읍 조곡리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1632,횡성군 횡성읍 청용리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1633,횡성군 횡성읍 추동리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1634,횡성군 횡성읍 학곡리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,0.999893,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [24]:
# 6. 조정된 점유율(단위유역별) 머지 및 결측 0 처리
share = share.merge(
    region_info, 
    on=['동리코드', '단위유역'], 
    how='left'
)
share['점유율'] = share['점유율'].fillna(0)

share

,동리코드,단위유역,시군,생활계,축산계,산업계,양식계,전,답,임야,...,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,점유율
0,강릉시 강동면 모전리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1,강릉시 강동면 산성우리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
2,강릉시 강동면 상시동리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
3,강릉시 강동면 심곡리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
4,강릉시 강동면 안인리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,횡성군 횡성읍 조곡리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0
1632,횡성군 횡성읍 청용리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,100.0
1633,횡성군 횡성읍 추동리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,1.000000,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,100.0
1634,횡성군 횡성읍 학곡리,섬강A,횡성군,1.0,1.0,1.0,0.0,1.0,1.0,0.999893,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,100.0


In [25]:
# 7. 점유율이 0인 경우에는 모든 계열을 0으로 설정
share.loc[share['점유율'] == 0, share_cols] = 0

In [26]:
# 8. 점유율이 100인 그룹에서 각 계열이 1 미만인 값은 1로 상향 조정
for col in share_cols:
    mask = (share['점유율'] == 100) & (share[col] < 1)
    share.loc[mask, col] = 1

share

,동리코드,단위유역,시군,생활계,축산계,산업계,양식계,전,답,임야,...,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,점유율
0,강릉시 강동면 모전리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1,강릉시 강동면 산성우리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
2,강릉시 강동면 상시동리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
3,강릉시 강동면 심곡리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
4,강릉시 강동면 안인리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,횡성군 횡성읍 조곡리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1632,횡성군 횡성읍 청용리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1633,횡성군 횡성읍 추동리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1634,횡성군 횡성읍 학곡리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0


In [27]:
# 9. 검증용 합계 테이블 생성
share_verif = (
    share
    .drop(columns='단위유역')
    .groupby(['동리코드', '시군'], as_index=False)[share_cols]
    .sum()
)

share_verif

,동리코드,시군,생활계,축산계,산업계,양식계,전,답,임야,대지,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지
0,강릉시 강동면 모전리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,강릉시 강동면 산성우리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,강릉시 강동면 상시동리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,강릉시 강동면 심곡리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,강릉시 강동면 안인리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,횡성군 횡성읍 정암리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1368,횡성군 횡성읍 조곡리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1369,횡성군 횡성읍 청용리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1370,횡성군 횡성읍 추동리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [28]:
# 10. 각 동리별 합계가 1 또는 0이 아닌 그룹에서는 '차이'만큼 점유율!=0인 행에 균등 분배
def distribute_diff(df):
    total = df[share_cols].sum()
    diff = 1 - total  # Series
    for col in share_cols:
        # Skip groups where the total is already ~1 or 0
        if abs(total[col] - 1) < 1e-8 or abs(total[col]) < 1e-8:
            continue
        d = diff[col]
        if abs(d) > 1e-8:
            mask = df['점유율'] != 0
            n = mask.sum()
            if n > 0:
                df.loc[mask, col] += d / n
    return df

share_distribute = (
    share
    .groupby(['동리코드', '시군'], group_keys=False)
    .apply(distribute_diff)
)

share_distribute

C:\Users\TG\AppData\Local\Temp\ipykernel_31376\3539633263.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(distribute_diff)


,동리코드,단위유역,시군,생활계,축산계,산업계,양식계,전,답,임야,...,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,점유율
0,강릉시 강동면 모전리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1,강릉시 강동면 산성우리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
2,강릉시 강동면 상시동리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
3,강릉시 강동면 심곡리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
4,강릉시 강동면 안인리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,횡성군 횡성읍 조곡리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1632,횡성군 횡성읍 청용리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1633,횡성군 횡성읍 추동리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1634,횡성군 횡성읍 학곡리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0


In [29]:
# 11. 각 계열의 동리코드 그룹별 합계가 0인 그룹은 각 계열 값을 '점유율 값 / 100'으로 적용

# 1) zero‐init 함수 정의
def init_zero_groups(df):
    """
    동리코드+시군 그룹 내에서
    각 col(생활계~유원지)의 합계가 0인 경우에만
    df[col] = df['점유율'] / 100 으로 초기화
    """
    total = df[share_cols].sum(axis=0)
    for col in share_cols:
        if abs(total[col]) < 1e-8:           # 그룹 합계가 0이면
            df[col] = df['점유율'] / 100     # 점유율(%)을 /100 하여 대입
    return df

# 2) 그룹키(동리코드+시군)로 묶어 함수 적용
share_distribute = (
    share_distribute
    .groupby(['동리코드', '시군'], group_keys=False)
    .apply(init_zero_groups)
)


share_distribute

C:\Users\TG\AppData\Local\Temp\ipykernel_31376\2376247098.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(init_zero_groups)


,동리코드,단위유역,시군,생활계,축산계,산업계,양식계,전,답,임야,...,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,점유율
0,강릉시 강동면 모전리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1,강릉시 강동면 산성우리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
2,강릉시 강동면 상시동리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
3,강릉시 강동면 심곡리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
4,강릉시 강동면 안인리,기타,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,횡성군 횡성읍 조곡리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1632,횡성군 횡성읍 청용리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1633,횡성군 횡성읍 추동리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1634,횡성군 횡성읍 학곡리,섬강A,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0


In [30]:
# 12. 분배 후 재검증
share_verif_after = (
    share_distribute
    .drop(columns='단위유역')
    .groupby(['동리코드', '시군'], as_index=False)
    .sum()
)

share_verif_after

,동리코드,시군,생활계,축산계,산업계,양식계,전,답,임야,대지,과수원,기타초지,기타,공장용지,공공시설지역,교통지역,주유소용지,체육용지,유원지,점유율
0,강릉시 강동면 모전리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1,강릉시 강동면 산성우리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
2,강릉시 강동면 상시동리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
3,강릉시 강동면 심곡리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
4,강릉시 강동면 안인리,강릉시,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,횡성군 횡성읍 정암리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1368,횡성군 횡성읍 조곡리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1369,횡성군 횡성읍 청용리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
1370,횡성군 횡성읍 추동리,횡성군,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0


In [31]:
# 13. share_distribute 엑셀로 저장
share_distribute.to_excel("E:/Coding/TMDL/전국오염원조사/단위유역별_점유율_조정.xlsx", index=False)

In [32]:
# 14. share_verif_after 엑셀로 저장
share_verif_after.to_excel("E:/Coding/TMDL/전국오염원조사/단위유역별_점유율_검증.xlsx", index=False)